In [1]:
import tensorflow as tf
import numpy as np
import sklearn
import random,os,io
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Dense
from tensorflow.keras.callbacks import Callback, EarlyStopping, LearningRateScheduler, ModelCheckpoint,CSVLogger
from tensorboard.plugins.hparams import api as hp

In [3]:
import pandas as pd
dep_train=pd.read_csv("D:\\bro\\Analysis\\mSANN\\cd\\XTr.dat",sep='\s+',names=[str(i) for i in range(0,16)])
indep_train=pd.read_csv("D:\\bro\\Analysis\\mSANN\\cd\\yTr.dat",sep='\s+',names=['target'])
dep_val=pd.read_csv("D:\\bro\\Analysis\\mSANN\\cd\\XV.dat",sep='\s+',names=[str(i) for i in range(0,16)])
indep_val=pd.read_csv("D:\\bro\\Analysis\\mSANN\\cd\\yV.dat",sep='\s+',names=["target"])
dep_test=pd.read_csv("D:\\bro\\Analysis\\mSANN\\cd\\XT.dat",sep='\s+',names=[str(i) for i in range(0,16)])
indep_test=pd.read_csv("D:\\bro\\Analysis\\mSANN\\cd\\yT.dat",sep='\s+',names=["target"])

dep_train.shape,indep_train.shape,dep_val.shape,indep_val.shape,dep_test.shape,indep_test.shape

((42039, 16), (42039, 1), (10935, 16), (10935, 1), (10934, 16), (10934, 1))

In [4]:
class SobolevNetwork(Model):
    def __init__(self, input_dim, num_hidden,init = None):
        super(SobolevNetwork, self).__init__()
        self.input_dim = input_dim 
        self.num_hidden = num_hidden
        self.W1 = tf.Variable(tf.random.normal([self.input_dim, self.num_hidden],stddev=0.1))
        self.b1 = tf.Variable(tf.ones([self.num_hidden]))
        self.W2 = tf.Variable(tf.random.normal([self.num_hidden, self.num_hidden],stddev=0.1))
        self.b2 = tf.Variable(tf.ones([self.num_hidden]))
        self.W3 = tf.Variable(tf.random.normal([self.num_hidden, self.num_hidden],stddev=0.1))
        self.b3 = tf.Variable(tf.ones([self.num_hidden]))
        self.W4 = tf.Variable(tf.random.normal([self.num_hidden, self.num_hidden],stddev=0.1))
        self.b4 = tf.Variable(tf.ones([self.num_hidden]))
        self.W5 = tf.Variable(tf.random.normal([self.num_hidden, self.num_hidden],stddev=0.1))
        self.b5 = tf.Variable(tf.ones([self.num_hidden]))
        self.W6 = tf.Variable(tf.random.normal([self.num_hidden, self.num_hidden],stddev=0.1))
        self.b6 = tf.Variable(tf.ones([self.num_hidden]))        
        self.W7 = tf.Variable(tf.random.normal([self.num_hidden, 1],stddev=0.1))
        self.b7 = tf.Variable(tf.ones([1]))
        self.w = [(self.W1, self.b1), (self.W2, self.b2), (self.W3, self.b3),(self.W4, self.b4), (self.W5, self.b5), (self.W6, self.b6),(self.W7, self.b7)]
        
    def call(self, X):
        #Input layer
        out = X
        #Hidden layers
        W,b = self.w[0]
        out = tf.nn.tanh(tf.matmul(out, W) + b)
        W,b = self.w[1]
        out = tf.nn.tanh(tf.matmul(out, W) + b)
        W,b = self.w[2]
        out = tf.nn.sigmoid(tf.matmul(out, W) + b)
        W,b = self.w[3]
        out = tf.nn.sigmoid(tf.matmul(out, W) + b)
        W,b = self.w[4]
        out = tf.nn.leaky_relu(tf.matmul(out, W) + b)
        W,b = self.w[5]
        out = tf.nn.relu(tf.matmul(out, W) + b)
        #Output layer
        W,b = self.w[-1]
        out = tf.matmul(out, W) + b
        return out


In [5]:
class LossCallback(Callback):
    def on_epoch_end(self, epoch, logs):
        print("\n For Epoch Number {} the model has loss[mean_absolute_error] of {}".format(epoch+1, logs["loss"]))

    def on_batch_end(self, batch, logs):
        print("\n For Batch Number {} the model has loss[mean_absolute_error] of {}".format(batch+1, logs))

In [6]:
csv_callback = CSVLogger( 'logs.csv',separator=',',append=True)

In [7]:
es_callback = EarlyStopping( monitor='val_loss',min_delta=0,patience=2, verbose=1, mode="auto", baseline=None, restore_best_weights=False)

In [8]:
def schedular(epoch, lr):
    if epoch <= 3:
        learning_rate = lr
    else:
        learning_rate = lr * tf.math.exp(-0.1)
        learning_rate = learning_rate.numpy()
    

    # with train_writer.as_default():
    #     tf.summary.scalar('LearningRateScheduler', data= learning_rate, step= epoch)
    
    return learning_rate

schedular_callback = LearningRateScheduler(schedular, verbose=1)

In [9]:
num_hidden_1 = 12

In [10]:
custom_snn= SobolevNetwork(input_dim=16,num_hidden=num_hidden_1)

In [11]:
custom_snn.compile(optimizer=tf.keras.optimizers.Adam(),  
            loss='mean_absolute_error')

In [16]:
import datetime
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)


In [17]:
custom_snn.fit(dep_train, indep_train, validation_data=(dep_val,indep_val),batch_size = 1000, epochs = 8, callbacks=[schedular_callback,LossCallback(), csv_callback,tensorboard_callback])


Epoch 1: LearningRateScheduler setting learning rate to 0.0006703201215714216.
Epoch 1/8

 For Batch Number 1 the model has loss[mean_absolute_error] of {'loss': 0.0021293265745043755}
 1/43 [..............................] - ETA: 16s - loss: 0.0021
 For Batch Number 2 the model has loss[mean_absolute_error] of {'loss': 0.0020510097965598106}

 For Batch Number 3 the model has loss[mean_absolute_error] of {'loss': 0.0020535450894385576}

 For Batch Number 4 the model has loss[mean_absolute_error] of {'loss': 0.002038370817899704}

 For Batch Number 5 the model has loss[mean_absolute_error] of {'loss': 0.0020624068565666676}

 For Batch Number 6 the model has loss[mean_absolute_error] of {'loss': 0.002063472056761384}

 For Batch Number 7 the model has loss[mean_absolute_error] of {'loss': 0.0020665598567575216}
 7/43 [===>..........................] - ETA: 0s - loss: 0.0021 
 For Batch Number 8 the model has loss[mean_absolute_error] of {'loss': 0.002076128264889121}

 For Batch Numbe

In [13]:
custom_snn.evaluate(dep_test, indep_test)

342/342 [==============================] - 2s 4ms/step - loss: 0.0022


0.002203913638368249

In [15]:
custom_snn.predict(dep_test)

342/342 [==============================] - 2s 5ms/step


array([[0.0103789 ],
       [0.01052278],
       [0.01038009],
       ...,
       [0.01052535],
       [0.01054573],
       [0.01016152]], dtype=float32)

In [21]:
%load_ext tensorboard
%tensorboard --logdir logs/fit


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 17416), started 0:06:31 ago. (Use '!kill 17416' to kill it.)

In [20]:
pip install jupyter-tensorboard


  Created wheel for jupyter-tensorboard: filename=jupyter_tensorboard-0.2.0-py2.py3-none-any.whl size=15250 sha256=8491c90c641169a2c30aa22d51d76772dd27332499bda808bd26bf1f646a783f
  Stored in directory: c:\users\kiran.franklin\appdata\local\pip\cache\wheels\98\03\14\670a2eefd86be0e21a529c469cadc2148cbeb00d82937e5aa2
Successfully built jupyter-tensorboard
